In [6]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import hashlib
from pathlib import Path
from urllib.request import urlretrieve as download
from gazpacho import Soup

# Pass the headers you want to retrieve from the xml such as ["loc", "lastmod"]
def parse_sitemap(url, headers):
    
    resp = requests.get(url)
    # we didn't get a valid response, bail
    if (200 != resp.status_code):
        return False

    # BeautifulSoup to parse the document
    soup = bs(resp.content, "xml")

    # find all the <url> tags in the document
    urls = soup.findAll('url')
    sitemaps = soup.findAll('sitemap')
    new_list = ["Source"] + headers
    panda_out_total = pd.DataFrame([], columns=new_list)


    if not urls and not sitemaps:
        return False

    # Recursive call to the the function if sitemap contains sitemaps
    if sitemaps:
        for u in sitemaps:
            sitemap_url = u.find('loc').string
            panda_recursive = parse_sitemap(sitemap_url, headers)
            panda_out_total = pd.concat([panda_out_total, panda_recursive], ignore_index=True)

    # storage for later...
    out = []

    # Creates a hash of the parent sitemap
    hash_sitemap = hashlib.md5(str(url).encode('utf-8')).hexdigest()

    # Extract the keys we want
    for u in urls:
        values = [hash_sitemap]
        for head in headers:
            loc = None
            loc = u.find(head)
            if not loc:
                loc = "None"
            else:
                loc = loc.string
            values.append(loc)
        out.append(values)
    
    # Creates a dataframe
    panda_out = pd.DataFrame(out, columns= new_list)

    # If recursive then merge recursive dataframe
    if not panda_out_total.empty:
        panda_out = pd.concat([panda_out, panda_out_total], ignore_index=True)

    #returns the dataframe
    return panda_out



In [8]:
site_links_df = parse_sitemap("https://www.psychobunny.com/sitemap.xml", ["loc"])

In [ ]:
pd.DataFrame([[a, b, c]], columns=['a', 'b', 'c'])

In [20]:
column_names = ["Image_Name", "URL"]

image_url_df = pd.DataFrame()

for url in site_links_df["loc"]:
    
    soup = Soup.get(url)

    image_containers = soup.find('img', {'class':'Image--lazyLoad Image--fadeIn'})

    try:
        for image in image_containers:
            for t in image.attrs:
                if t == "src":
                    image_url = "https:" + image.attrs[t]
                    image_name = url.split("/")[-1]
                    to_concat = pd.DataFrame([[image_name, image_url]], columns = column_names)

                    if image_url_df.empty:
                        image_url_df = to_concat                    
                    else:
                        image_url_df = pd.concat([image_url_df, to_concat])

                elif t == "data-original-src":
                    image_url = "https:" + image.attrs[t]
                    image_name = url.split("/")[-1]
                    to_concat = pd.DataFrame([[image_name, image_url]], columns = column_names)

                    if image_url_df.empty:
                        image_url_df = to_concat                    
                    else:
                        image_url_df = pd.concat([image_url_df, to_concat])
    except Exception as e:
        print(e)
        pass

'Soup' object is not iterable
'Soup' object is not iterable
'Soup' object is not iterable
'Soup' object is not iterable
'Soup' object is not iterable
'Soup' object is not iterable
'Soup' object is not iterable
'NoneType' object is not iterable
'Soup' object is not iterable
'Soup' object is not iterable
'Soup' object is not iterable
'Soup' object is not iterable
'Soup' object is not iterable
'Soup' object is not iterable
'NoneType' object is not iterable
'Soup' object is not iterable
'Soup' object is not iterable
'Soup' object is not iterable
'Soup' object is not iterable
'NoneType' object is not iterable
'NoneType' object is not iterable
'NoneType' object is not iterable
'NoneType' object is not iterable
'NoneType' object is not iterable
'NoneType' object is not iterable
'Soup' object is not iterable
'Soup' object is not iterable
'Soup' object is not iterable
'Soup' object is not iterable
'Soup' object is not iterable
'Soup' object is not iterable
'NoneType' object is not iterable
'Non

In [21]:
image_url_df

,Image_Name,URL
0,mens-big-and-tall-classic-polo-b9k001arpc-001-...,https://cdn.shopify.com/s/files/1/2304/1219/pr...
0,mens-big-and-tall-classic-polo-b9k001arpc-001-...,https://cdn.shopify.com/s/files/1/2304/1219/pr...
0,mens-big-and-tall-classic-polo-b9k001arpc-001-...,https://cdn.shopify.com/s/files/1/2304/1219/pr...
0,mens-big-and-tall-classic-polo-b9k001arpc-001-...,https://cdn.shopify.com/s/files/1/2304/1219/pr...
0,mens-big-and-tall-classic-polo-b9k001arpc-001-...,https://cdn.shopify.com/s/files/1/2304/1219/pr...
...,...,...
0,kids-carson-tee-bou329s1pc-001-black,https://cdn.shopify.com/s/files/1/2304/1219/pr...
0,kids-carson-tee-bou329s1pc-001-black,https://cdn.shopify.com/s/files/1/2304/1219/pr...
0,kids-carson-tee-bou329s1pc-001-black,https://cdn.shopify.com/s/files/1/2304/1219/pr...
0,kids-carson-tee-bou329s1pc-001-black,https://cdn.shopify.com/s/files/1/2304/1219/pr...


In [22]:
image_url_df.to_csv("data/images_to_rec.csv")

In [17]:
site_links_df = parse_sitemap("https://www.psychobunny.com/sitemap.xml", ["loc"])

TypeError: __init__() takes from 1 to 2 positional arguments but 3 were given

In [16]:
site_links_df["loc"][550].split("/")[-1]

'mens-bailey-sweat-shorts-b6r518r1ft-410-navy'

In [192]:
from pathlib import Path
from urllib.request import urlretrieve as download
from gazpacho import Soup

# url = 'https://www.psychobunny.com/collections/swimwear-bathing-suits/products/mens-everett-swim-trunks-b6w326s1po675-love-pink?nosto_source=cmp&nosto=6245b2db9e91740d7eab687a'
id = 1
for url in site_links_df["loc"]:
    
    soup = Soup.get(url)

    image_containers = soup.find('img', {'class':'Image--lazyLoad Image--fadeIn'})

    for image in image_containers:
        for t in image.attrs:
            if t == "src":
                image_url = "https:" + image.attrs[t]
                #print(image.attrs[t])
                directory = "images"
                Path(directory).mkdir(exist_ok=True)
                download(image_url, f"{directory}/{str(id)}.jpg")
                id += 1
            
            elif t == "data-original-src":
                print(image.attrs[t])
                

TypeError: 'Soup' object is not iterable

# APPENDEX

In [119]:
image = soup.find('img')

In [141]:
image[0].find('img').tag(data-src)

NameError: name 'data' is not defined

In [122]:
directory = "images"
Path(directory).mkdir(exist_ok=True)

extension = image.split('.')[-1]
download(image, f"{directory}/{id}.{extension}")

AttributeError: 'list' object has no attribute 'split'

In [152]:
image

<img src="https://cdn.shopify.com/s/files/1/2304/1219/files/Curalate_PDP.jpg">

In [ ]:
import requests
from bs4 import BeautifulSoup

In [153]:
url = 'https://www.psychobunny.com/collections/swimwear-bathing-suits/products/mens-everett-swim-trunks-b6w326s1po675-love-pink?nosto_source=cmp&nosto=6245b2db9e91740d7eab687a'
html = requests.get(url).text

In [154]:
soup = BeautifulSoup(html, 'html.parser')

In [162]:
image = soup.find('img')

In [163]:
image

[<img class="lazyload" data-src="https://cdn.shopify.com/s/files/1/2304/1219/files/update_your_browser_popup_V1_03.jpg?v=1579636488">,
 <img class="lazyload" data-src="https://cdn.shopify.com/s/files/1/2304/1219/files/update_your_browser_popup_V1_05.jpg?v=1579636495">,
 <img class="lazyload" data-src="https://cdn.shopify.com/s/files/1/2304/1219/files/update_your_browser_popup_V1_09.jpg?v=1579636503">,
 <img src="https://cdn.shopify.com/s/files/1/2304/1219/files/sticky_nav_desktop_logo_48.png?v=1582739685" style="width: 50%;">,
 <img class="Header__LogoImage Header__LogoImage--primary" src="//cdn.shopify.com/s/files/1/2304/1219/files/new-logo_x4_fabba486-3288-48c4-80d6-dfb090d4e068_200x.png?v=1623664375" srcset="//cdn.shopify.com/s/files/1/2304/1219/files/new-logo_x4_fabba486-3288-48c4-80d6-dfb090d4e068_200x.png?v=1623664375 1x, //cdn.shopify.com/s/files/1/2304/1219/files/new-logo_x4_fabba486-3288-48c4-80d6-dfb090d4e068_200x@2x.png?v=1623664375 2x" width="200" alt="Psycho Bunny">,
 <img

In [157]:
image = (soup
    .find("img")
    # .attrs['src']
)

# image = (soup
#     .find("div", {"id": "title-overview"})
#     .find("div", {"class": "poster"})
#     .find("img")
#     .attrs['src']
# )

In [158]:
test = soup.find({'class':'img'})
test

In [159]:
type(test)

NoneType

In [160]:
print(image[0].find("data-src"))

KeyError: 0

In [57]:
from PIL import Image
import requests
import bs4

url = 'https://www.psychobunny.com/collections/swimwear-bathing-suits/products/mens-everett-swim-trunks-b6w326s1po675-love-pink?nosto_source=cmp&nosto=6245b2db9e91740d7eab687a'

response = requests.get(url)

soup = bs4.BeautifulSoup(response.text, 'html.parser')

image = soup.find('img')
print(image)
image_url = image['data-original-src']

img = Image.open(requests.get(image_url, stream = True).raw)

img.save('image.jpg')

<img class="lazyload" data-src="https://cdn.shopify.com/s/files/1/2304/1219/files/update_your_browser_popup_V1_03.jpg?v=1579636488"/>


KeyError: 'data-original-src'

In [59]:
soup = BeautifulSoup(urllib2.urlopen(url).read())
link = soup.find(itemprop="image")
print(link["src"])

NameError: name 'urllib2' is not defined

In [58]:
image

<img class="lazyload" data-src="https://cdn.shopify.com/s/files/1/2304/1219/files/update_your_browser_popup_V1_03.jpg?v=1579636488"/>

In [ ]:
<img class="Image--fadeIn lazyautosizes Image--lazyLoaded" src="//cdn.shopify.com/s/files/1/2304/1219/products/B6W326S1PO-LPI_2_250x.jpg?v=1648034484" data-widths="[200,400,600,700,800,900,1000,1200,1400]" data-sizes="auto" data-expand="-100" alt="MENS EVERETT SWIM TRUNKS - B6W326S1PO - 675 Love Pink" data-max-width="1500" data-max-height="1870" data-original-src="//cdn.shopify.com/s/files/1/2304/1219/products/B6W326S1PO-LPI_2.jpg?v=1648034484" data-srcset="//cdn.shopify.com/s/files/1/2304/1219/products/B6W326S1PO-LPI_2_200x.jpg?v=1648034484 200w, //cdn.shopify.com/s/files/1/2304/1219/products/B6W326S1PO-LPI_2_400x.jpg?v=1648034484 400w, //cdn.shopify.com/s/files/1/2304/1219/products/B6W326S1PO-LPI_2_600x.jpg?v=1648034484 600w, //cdn.shopify.com/s/files/1/2304/1219/products/B6W326S1PO-LPI_2_700x.jpg?v=1648034484 700w, //cdn.shopify.com/s/files/1/2304/1219/products/B6W326S1PO-LPI_2_800x.jpg?v=1648034484 800w, //cdn.shopify.com/s/files/1/2304/1219/products/B6W326S1PO-LPI_2_900x.jpg?v=1648034484 900w, //cdn.shopify.com/s/files/1/2304/1219/products/B6W326S1PO-LPI_2_1000x.jpg?v=1648034484 1000w, //cdn.shopify.com/s/files/1/2304/1219/products/B6W326S1PO-LPI_2_1200x.jpg?v=1648034484 1200w, //cdn.shopify.com/s/files/1/2304/1219/products/B6W326S1PO-LPI_2_1400x.jpg?v=1648034484 1400w" sizes="931px" srcset="//cdn.shopify.com/s/files/1/2304/1219/products/B6W326S1PO-LPI_2_200x.jpg?v=1648034484 200w, //cdn.shopify.com/s/files/1/2304/1219/products/B6W326S1PO-LPI_2_400x.jpg?v=1648034484 400w, //cdn.shopify.com/s/files/1/2304/1219/products/B6W326S1PO-LPI_2_600x.jpg?v=1648034484 600w, //cdn.shopify.com/s/files/1/2304/1219/products/B6W326S1PO-LPI_2_700x.jpg?v=1648034484 700w, //cdn.shopify.com/s/files/1/2304/1219/products/B6W326S1PO-LPI_2_800x.jpg?v=1648034484 800w, //cdn.shopify.com/s/files/1/2304/1219/products/B6W326S1PO-LPI_2_900x.jpg?v=1648034484 900w, //cdn.shopify.com/s/files/1/2304/1219/products/B6W326S1PO-LPI_2_1000x.jpg?v=1648034484 1000w, //cdn.shopify.com/s/files/1/2304/1219/products/B6W326S1PO-LPI_2_1200x.jpg?v=1648034484 1200w, //cdn.shopify.com/s/files/1/2304/1219/products/B6W326S1PO-LPI_2_1400x.jpg?v=1648034484 1400w">

In [41]:
from bs4 import BeautifulSoup
import urllib.request
import requests

# setting URL destination 
url = site_links_df["loc"][0]

# retrieving HTML payload from the website
response = requests.get(url)

# using BeautifulSoup to parse the response object
soup = BeautifulSoup(response.content, "html.parser")

# finding Post images in the soup
images = soup.find_all("img src", attrs={"alt":"Post image"})

print(images)

# downloading images
number = 0
for image in images:
    print(image["img src"])
    image_src = image["img src"]
    urllib.request.urlretrieve(image_src, str(number))
    number += 1

[]


In [46]:
from bs4 import *
import requests
import os
 
# CREATE FOLDER
def folder_create(images):
    try:
        folder_name = input("Enter Folder Name:- ")
        # folder creation
        os.mkdir(folder_name)
 
    # if folder exists with that name, ask another name
    except:
        print("Folder Exist with that name! Enter New Name")
        folder_name = input("Enter Folder Name:- ")
        folder_create()
 
    # image downloading start
    download_images(images, folder_name)
 
 
# DOWNLOAD ALL IMAGES FROM THAT URL
def download_images(images, folder_name):
   
    # initial count is zero
    count = 0
 
    # print total images found in URL
    print(f"Total {len(images)} Image Found!")
 
    # checking if images is not zero
    if len(images) != 0:
        for i, image in enumerate(images):
            # From image tag ,Fetch image Source URL
 
                        # 1.data-srcset
                        # 2.data-src
                        # 3.data-fallback-src
                        # 4.src
 
            # Here we will use exception handling
 
            # first we will search for "data-srcset" in img tag
            try:
                # In image tag ,searching for "data-srcset"
                image_link = image["srcset"]
                 
            # then we will search for "data-src" in img
            # tag and so on..
            except:
                try:
                    # In image tag ,searching for "data-src"
                    image_link = image["src"]
                except:
                    try:
                        # In image tag ,searching for "data-fallback-src"
                        image_link = image["data-fallback-src"]
                    except:
                        try:
                            # In image tag ,searching for "src"
                            image_link = image["src"]
 
                        # if no Source URL found
                        except:
                            pass
 
            # After getting Image Source URL
            # We will try to get the content of image
            try:
                r = requests.get(image_link).content
                try:
 
                    # possibility of decode
                    r = str(r, 'utf-8')
 
                except UnicodeDecodeError:
 
                    # After checking above condition, Image Download start
                    with open(f"{folder_name}/images{i+1}.jpg", "wb+") as f:
                        f.write(r)
 
                    # counting number of image downloaded
                    count += 1
            except:
                pass
 
        # There might be possible, that all
        # images not download
        # if all images download
        if count == len(images):
            print("All Images Downloaded!")
             
        # if all images not download
        else:
            print(f"Total {count} Images Downloaded Out of {len(images)}")
 
# MAIN FUNCTION START
def main(url):
   
    # content of URL
    r = requests.get(url)
 
    # Parse HTML Code
    soup = BeautifulSoup(r.text, 'html.parser')
 
    # find all images in URL
    images = soup.findAll('img')
 
    # Call folder create function
    folder_create(images)
 
 
# take url
url = input("Enter URL:- ")
 
# CALL MAIN FUNCTION
main(url)

Enter URL:-  https://www.psychobunny.com/collections/swimwear-bathing-suits/products/mens-everett-swim-trunks-b6w326s1po675-love-pink?nosto_source=cmp&nosto=6245b2db9e91740d7eab687a
Enter Folder Name:-  st


Total 19 Image Found!
Total 2 Images Downloaded Out of 19


In [ ]:
#### Scrape and Download Images from parent site URL

In [7]:
site_links_df['loc'][0]

'https://www.psychobunny.com/'